In [1]:
import gensim
import keras

c:\users\paul\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format ('models/GoogleNews-vectors-negative300.bin', binary=True)  

In [3]:
model = keras.models.load_model("models/bilstm_cos.h5")

In [4]:
def detect_arguments(model, post):
    # split text into sentences
    post_sentences = post.split(".")
    # count amount of sentences
    n_sentences = len(post_sentences)
    # add topic
    topics = pd.Series(["no topic"] * (n_sentences), name = "topic")
    # make data ready for encoding
    sentences = pd.DataFrame(post_sentences)
    df = pd.concat([sentences, topics], axis = 1)
    df = df.rename(index=str, columns={0: "sentence"})
    encoded_sentences, _ =  embed_sentences(sentences, len(sentences), topics)
    
    arguments = []
    non_arguments = []
    
    # feed sentences into LSTM and get prediction
    for i in range(len(encoded_sentences)):
        n_words = encoded_sentences[i].shape[0]
        if(n_words > 0):
            n_features = encoded_sentences[i].shape[1]
            prediction = model.predict(encoded_sentences[i].reshape(1, n_words, n_features), batch_size=1, verbose=0)
            if(prediction > 0.5):
                arguments.append((prediction, post_sentences[i]))
            else:
                non_arguments.append((prediction, post_sentences[i]))
    return arguments, non_arguments      